In [149]:
import pandas as pd
import numpy as np
import re
import datetime
import time
import random

def fun_data_clean(dir_str, dt):
    data = pd.read_excel(dir_str)

    def phone_num(x):
        phone_list = re.findall('\d.\d{3}.\d{3}.\d{2}.\d{2}',x)
        phone_list = [re.sub('\D','',x) for x in phone_list]
        if len(phone_list) == 0:
            return [-1,-1]
        else:
            phone_list = [int(x) for x in phone_list]
        if len(phone_list) == 1:
            phone_list.append(-1)
        return phone_list

    def time_posting(x):
        if pd.isnull(x):
            return datetime.datetime(3000,1,1,0,0)
        hour = int(re.findall(', (\d+):\d+',x)[0])
        minute = int(re.findall(', \d+:(\d+)',x)[0])
        s = re.split(',', x)[0]
        if s == u'вчера':
            dt_tmp = dt + datetime.timedelta(-1)
            day = dt_tmp.day
            month = dt_tmp.month
        elif s == u'сегодня': 
            day = dt.day
            month = dt.month
        else:
            s = re.sub(u'янв', '01', s.lower())
            s = re.sub(u'фев', '02', s.lower())
            s = re.sub(u'мар', '03', s.lower())
            s = re.sub(u'апр', '04', s.lower())
            s = re.sub(u'май', '05', s.lower())
            s = re.sub(u'июн', '06', s.lower())
            s = re.sub(u'июл', '07', s.lower())
            s = re.sub(u'авг', '08', s.lower())
            s = re.sub(u'сен', '09', s.lower())
            s = re.sub(u'окт', '10', s.lower())
            s = re.sub(u'ноя', '11', s.lower())
            s = re.sub(u'дек', '12', s.lower())
            day = int(re.findall('(\d+) \d+', s)[0])
            month = int(re.findall('\d+ (\d+)', s)[0])
        year = datetime.date.today().year
        return datetime.datetime(year,month,day,hour,minute)

    cnt_rooms = data.cnt_rooms.apply(lambda x: -1 if len(re.findall('^\d',x))==0
                                                     else int(re.findall('^\d',x)[0]))
    flat_ids = data.url.apply(lambda x: -1 if len(re.findall('/(\d+)/',x))==0
                                            else int(re.findall('/(\d+)/',x)[0]))

    data.dt_metro.fillna('NULL',inplace=True)
    dt_metro_int = data.dt_metro.apply(lambda x: -1 if len(re.findall('^\d+', re.sub(' +', '', x)))==0
                                                    else int(re.findall('^\d+', re.sub(' +', '', x))[0]))
    dt_metro_type =  data.dt_metro.apply(lambda x: "NULL" if len(re.findall('^\d+\D+', re.sub(' +','',x)))==0
                                                             else re.findall('^\d+(\D+)', re.sub(' +','',x))[0])
    data.popularity.fillna("NULL",inplace=True)
    popularity_all = data.popularity.apply(lambda x: -1 if len(re.findall(u'всего (\d+)',x))==0 
                                                           else int(re.findall(u'всего (\d+)',x)[0]))
    popularity_today = data.popularity.apply(lambda x: -1 if len(re.findall(u'сегодня (\d+)',x))==0
                                                             else int(re.findall(u'сегодня (\d+)',x)[0]))
    floor_cur = data.floor.apply(lambda x: -1 if len(re.findall('/',x))==0
                                                             else int(re.split('/',x)[0]))
    floor_all = data.floor.apply(lambda x: -1 if len(re.findall('/',x))==0
                                                             else int(re.split('/',x)[1]))
    living_area = data.living_area.apply(lambda x: -1 if len(re.findall("\D",re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',x)))))>2
                                                             else float(re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',x)))))
    kitchen_area = data.kitchen_area.apply(lambda x: -1 if len(re.findall("\D",re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',x)))))>2
                                                             else float(re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',x)))))

    for f in ['avg_price_m2_in_house',
              'avg_rental_rate',
              'district_population',
              'avg_age',
              'avg_price_m2_in_district']:
        data[f].fillna('0',inplace=True)
    avg_price_m2_in_house = data.avg_price_m2_in_house.apply(lambda x: int(re.sub('\D','',x)))
    avg_rental_rate = data.avg_rental_rate.apply(lambda x: int(re.sub('\D','',x)))
    district_population = data.district_population.apply(lambda x: int(re.sub('\D','',x)))
    avg_age = data.avg_age.apply(lambda x: int(re.sub('\D','',x)))
    avg_price_m2_in_district = data.avg_price_m2_in_district.apply(lambda x: int(re.sub('\D','',x)))
    phone_first = [x[0] for x in data.message.apply(phone_num)]
    phone_second = [x[1] for x in data.message.apply(phone_num)]

    data_clean = pd.DataFrame(data=zip(flat_ids,
                              cnt_rooms,
                              data.total_area,
                              kitchen_area,
                              living_area,
                              floor_cur,
                              floor_all,
                              data.price_all,
                              np.round((data.price_all/data.total_area).values),
                              avg_price_m2_in_house,
                              avg_price_m2_in_district,
                              avg_rental_rate,
                              popularity_today,
                              popularity_all,
                              data.metro,
                              dt_metro_int,
                              dt_metro_type,
                              data.year_of_construction,
                              data.house_type,
                              data.cnt_entrance,
                              data.cnt_flats,
                              data.name_of_district,
                              district_population,
                              avg_age,
                              phone_first,
                              phone_second,
                              data.time_of_posting.apply(time_posting),           
                              [dt for i in range(data.shape[0])]  
                             ),
                     columns = ['id',
                                'cnt_rooms',
                                'total_area',
                                'kitchen_area',
                                'living_area',
                                'floor_cur',
                                'floor_all',
                                'price_all',
                                'price_m2',
                                'avg_price_m2_in_house',
                                'avg_price_m2_in_district',
                                'avg_rental_rate',
                                'popularity_today',
                                'popularity_all',
                                'metro',
                                'metro_time',
                                'metro_time_type',
                                'year_of_construction',
                                'house_type',
                                'cnt_entrance',
                                'cnt_flats',
                                'name_of_district',
                                'district_population',
                                'avg_age_of_constructions',
                                'phone_first',
                                'phone_second',
                                'dt_post',
                                'dt_report'
                               ]
                    )
    return data_clean

In [ ]:
data.ix[79,:]

In [ ]:
data_clean.ix[79,:]

In [150]:
path_str = 'C:\\Users\\vvashchenko\\Desktop\\kaggle\\cian'

zyablikovo_060916 = fun_data_clean(dir_str = path_str+r'\zyablikovo_060916.xlsx',dt=datetime.date(2016,9,6))
zyablikovo_110916 = fun_data_clean(dir_str = path_str+r'\zyablikovo_110916.xlsx',dt=datetime.date(2016,9,11))

orehovo_borisovo_yuzhnoe_060916 = fun_data_clean(dir_str = path_str+r'\orehovo_borisovo_yuzhnoe_060916.xlsx',dt=datetime.date(2016,9,6))
orehovo_borisovo_yuzhnoe_110916 = fun_data_clean(dir_str = path_str+r'\orehovo_borisovo_yuzhnoe_110916.xlsx',dt=datetime.date(2016,9,11))

orehovo_borisovo_severnoe_060916 = fun_data_clean(dir_str = path_str+r'\orehovo_borisovo_severnoe_060916.xlsx',dt=datetime.date(2016,9,6))
orehovo_borisovo_severnoe_110916 = fun_data_clean(dir_str = path_str+r'\orehovo_borisovo_severnoe_110916.xlsx',dt=datetime.date(2016,9,11))

nagatino_sadovniki_060916 = fun_data_clean(dir_str = path_str+r'\nagatino_sadovniki_060916.xlsx',dt=datetime.date(2016,9,6))
nagatino_sadovniki_110916 = fun_data_clean(dir_str = path_str+r'\nagatino_sadovniki_110916.xlsx',dt=datetime.date(2016,9,11))

moskvoreche_saburovo_060916 = fun_data_clean(dir_str = path_str+r'\moskvoreche_saburovo_060916.xlsx',dt=datetime.date(2016,9,6))
moskvoreche_saburovo_110916 = fun_data_clean(dir_str = path_str+r'\moskvoreche_saburovo_110916.xlsx',dt=datetime.date(2016,9,11))

caricyno_060916 = fun_data_clean(dir_str = path_str+r'\caricyno_060916.xlsx',dt=datetime.date(2016,9,6))
caricyno_110916 = fun_data_clean(dir_str = path_str+r'\caricyno_110916.xlsx',dt=datetime.date(2016,9,11))

brateevo_060916 = fun_data_clean(dir_str = path_str+r'\brateevo_060916.xlsx',dt=datetime.date(2016,9,6))
brateevo_110916 = fun_data_clean(dir_str = path_str+r'\brateevo_110916.xlsx',dt=datetime.date(2016,9,11))

In [151]:
frames = [zyablikovo_060916, zyablikovo_110916, 
          orehovo_borisovo_yuzhnoe_060916, orehovo_borisovo_yuzhnoe_110916,
          orehovo_borisovo_severnoe_060916, orehovo_borisovo_severnoe_110916,
          nagatino_sadovniki_060916, nagatino_sadovniki_110916,
          moskvoreche_saburovo_060916, moskvoreche_saburovo_110916,
          caricyno_060916, caricyno_110916,
          brateevo_060916, brateevo_110916]
result = pd.concat(frames)

In [30]:
result.ix[result.id == 149386843,['popularity_today','popularity_all']]

,popularity_today,popularity_all
0,1,7
390,6,22


In [28]:
result.index = range(result.shape[0])

In [33]:
len(np.unique(result.id))

3297

In [61]:
time.sleep(random.uniform(0,1.5))

In [71]:
grouped = result['id'].groupby(result['name_of_district'])
grouped.nunique()

name_of_district
Братеево                     456
Зябликово                    421
Москворечье-Сабурово         315
Нагатино-Садовники           391
Орехово-Борисово Северное    356
Орехово-Борисово Южное       484
Царицыно                     484
dtype: int64

In [72]:
grouped.count()

name_of_district
Братеево                     765
Зябликово                    750
Москворечье-Сабурово         561
Нагатино-Садовники           690
Орехово-Борисово Северное    620
Орехово-Борисово Южное       848
Царицыно                     855
dtype: int64

In [115]:
grouped = result.id.groupby([result.id,result.name_of_district])

In [119]:
grouped_1 = grouped.count()

In [136]:
grouped_2 = grouped_1.groupby([grouped_1.index.labels[1],grouped_1.values])

In [137]:
grouped_1.index.levels[1]

Index([u'Братеево', u'Зябликово', u'Москворечье-Сабурово',
       u'Нагатино-Садовники', u'Орехово-Борисово Северное',
       u'Орехово-Борисово Южное', u'Царицыно'],
      dtype='object', name=u'name_of_district')

In [138]:
grouped_2.sum()

0  1    147
   2    618
1  1     92
   2    658
2  1     69
   2    492
3  1     92
   2    598
4  1     92
   2    528
5  1    120
   2    728
6  1    113
   2    742
dtype: int64

In [154]:
res_tmp = result.price_m2/result.avg_price_m2_in_house 

In [165]:
np.unique(result.id[res_tmp<0.75].values)

array([  4441862,  47710547, 146815940, 148622506, 148811244, 148908314,
       148931398, 149140379, 149172418, 149326582, 149353414, 149516255,
       149516956, 149542229], dtype=int64)

In [167]:
zyablikovo_060916

,id,cnt_rooms,total_area,kitchen_area,living_area,floor_cur,floor_all,price_all,price_m2,avg_price_m2_in_house,...,house_type,cnt_entrance,cnt_flats,name_of_district,district_population,avg_age_of_constructions,phone_first,phone_second,dt_post,dt_report
0,149386843,3,58,8.0,42.0,8,9,8800000,151724,156384,...,панельный,6,215,Зябликово,132348,33,79653196975,74957770032,2016-09-04 14:09:00,2016-09-06
1,28139920,4,10,10.0,64.0,6,22,12400000,1240000,147425,...,панельный,2,174,Зябликово,132348,33,79035424224,-1,2016-09-02 17:34:00,2016-09-06
2,149232579,3,60,6.0,44.0,1,9,10800000,180000,156394,...,панельный,8,287,Зябликово,132348,33,74953630160,-1,2016-09-06 00:51:00,2016-09-06
3,146945483,5,10,17.2,61.0,1,14,10990000,1099000,156095,...,панельный,4,190,Зябликово,132348,33,79261132246,-1,2016-08-21 11:00:00,2016-09-06
4,22527282,2,59,10.0,36.0,21,22,9900000,167797,157484,...,панельный,3,261,Зябликово,132348,33,79167365429,79263733832,2016-08-24 12:33:00,2016-09-06
5,145522442,2,56,10.0,32.0,2,17,9000000,160714,164151,...,панельный,3,416,Зябликово,132348,33,79162749545,-1,2016-08-30 13:35:00,2016-09-06
6,147924231,3,76,11.0,51.0,2,17,12000000,157895,155290,...,панельный,3,203,Зябликово,132348,33,79660257520,-1,2016-07-01 20:31:00,2016-09-06
7,145240694,1,32,7.0,18.1,1,9,4800000,150000,155828,...,панельный,4,144,Зябликово,132348,33,74952320073,79645608716,2016-09-05 18:30:00,2016-09-06
8,149431421,3,62,9.0,40.0,4,12,9690000,156290,156095,...,панельный,4,190,Зябликово,132348,33,79166893818,-1,2016-09-06 13:06:00,2016-09-06
9,149245791,1,40,8.0,19.0,2,17,6100000,152500,165152,...,панельный,3,203,Зябликово,132348,33,74957850055,79067407623,2016-08-30 20:13:00,2016-09-06
